### Introduction To Data Ingestion

In [2]:
import os
from typing import List, Dict, Any
from langchain_core.documents import Document
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
)

/home/aniruddha/Projects/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Understanding Document Structure In Langchain

In [3]:
## Create a simple document
doc = Document(
    page_content="This is the main text content that will be embedded and searched.",
    metadata ={
        "source":"example.txt",
        "author":"Aniruddha",
        "date":"2025-12-12",
        "custom_field":"custom_value"
    }
)

print("Document structure")
print(f"content: {doc.page_content}")
print(f"metadata: {doc.metadata}")

Document structure
content: This is the main text content that will be embedded and searched.
metadata: {'source': 'example.txt', 'author': 'Aniruddha', 'date': '2025-12-12', 'custom_field': 'custom_value'}


In [4]:
type(doc)

langchain_core.documents.base.Document

### TextLoader- Read Single File 

In [7]:
from langchain_community.document_loaders import TextLoader

## Loading a single text file
loader = TextLoader(
    file_path="data/text_files/python_intro.txt",
    encoding="utf8"
)

documents = loader.load()
print(f"📄 Loaded {len(documents)} document")
print(f"First document content: {documents[0].page_content[:100]}...")
print(f"First document metadata: {documents[0].metadata}")

📄 Loaded 1 document
First document content: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
First document metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader- Multiple Text Files

In [9]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
   "data/text_files/",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf8"},
    show_progress=True
)

documents = dir_loader.load()
print(f"📁 Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")


# 📊 Analysis
print("\n📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print("  - Loads multiple files at once")
print("  - Supports glob patterns")
print("  - Progress tracking")
print("  - Recursive directory scanning")

print("\n❌ Disadvantages:")
print("  - All files must be same type")
print("  - Limited error handling per file")
print("  - Can be memory intensive for large directories")

100%|██████████| 2/2 [00:00<00:00, 1663.42it/s]

📁 Loaded 2 documents

Document 1:
  Source: data/text_files/machine_learning.txt
  Length: 575 characters

Document 2:
  Source: data/text_files/python_intro.txt
  Length: 489 characters

📊 DirectoryLoader Characteristics:
✅ Advantages:
  - Loads multiple files at once
  - Supports glob patterns
  - Progress tracking
  - Recursive directory scanning

❌ Disadvantages:
  - All files must be same type
  - Limited error handling per file
  - Can be memory intensive for large directories


### Text Splitting Statergies

In [10]:
print(documents)

[Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '), Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramm

In [20]:
# Method 1: Character-based splitting

print("1️⃣ CHARACTER TEXT SPLITTER")

char_splitter = CharacterTextSplitter(
    separator="\n", # Split on newlines
    chunk_size=200, # Max 200 characters per chunk
    chunk_overlap=20, # 50 characters overlap
    length_function=len # Use len() to measure text length
)

text=documents[0].page_content
char_chunks = char_splitter.split_text(text)
print(f"Created chunks: {len(char_chunks)}")
print(f"First chunk: {char_chunks[0][:100]}...")

1️⃣ CHARACTER TEXT SPLITTER
Created chunks: 4
First chunk: Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems...


In [21]:
print(char_chunks[0])
print("--------------------------------------------------")
print(char_chunks[1])

Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
--------------------------------------------------
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.
Types of Machine Learning:


In [ ]:
# Method 2: Recursive character splitting (RECOMMENDED)

print("\n2️⃣ RECURSIVE CHARACTER TEXT SPLITTER")
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], # Hierarchical separators
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)

### separators = ["\n\n", "\n", " ", ""], separators=[] # No splitting, return full text as one chunk, separators=None # Default separators

chunks = recursive_splitter.split_text(text)
print(f"Created chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:100]}...")
print(chunks)


2️⃣ RECURSIVE CHARACTER TEXT SPLITTER
Created chunks: 6
First chunk: Machine Learning Basics...
['Machine Learning Basics', 'Machine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs', 'that can access data and use it to learn for themselves.', 'Types of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data', '3. Reinforcement Learning: Learning through rewards and penalties', 'Applications include image recognition, speech processing, and recommendation systems']


In [24]:
print(chunks[0])
print("--------------------------------------------------")
print(chunks[1])
print("--------------------------------------------------")
print(chunks[2])

Machine Learning Basics
--------------------------------------------------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
--------------------------------------------------
that can access data and use it to learn for themselves.


In [27]:
# Method 3: Token-based splitting
import token


print("\n3️⃣ TOKEN TEXT SPLITTER")

token_splitter = TokenTextSplitter(
    chunk_size=50, # Max 200 tokens per chunk
    chunk_overlap=20, # 20 tokens overlap
    encoding_name="gpt2" # Use GPT-2 tokenizer
)

token_chunks = token_splitter.split_text(text)
print(f"Created chunks: {len(token_chunks)}")
print(f"First chunk: {token_chunks[0][:100]}...")


3️⃣ TOKEN TEXT SPLITTER
Created chunks: 4
First chunk: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system...


In [28]:
print(token_chunks[0])
print("--------------------------------------------------")
print(token_chunks[1])
print("--------------------------------------------------")
print(token_chunks[2])

Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types
--------------------------------------------------
 on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
--------------------------------------------------
: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


   


In [29]:
# 📊 Comparison
print("\n📊 Text Splitting Methods Comparison:")
print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  Use when: Working with token-limited models")


📊 Text Splitting Methods Comparison:

CharacterTextSplitter:
  ✅ Simple and predictable
  ✅ Good for structured text
  ❌ May break mid-sentence
  Use when: Text has clear delimiters

RecursiveCharacterTextSplitter:
  ✅ Respects text structure
  ✅ Tries multiple separators
  ✅ Best general-purpose splitter
  ❌ Slightly more complex
  Use when: Default choice for most texts

TokenTextSplitter:
  ✅ Respects model token limits
  ✅ More accurate for embeddings
  ❌ Slower than character-based
  Use when: Working with token-limited models
